In [30]:
import torch
from torch import Tensor
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

In [44]:
dataset = Planetoid(root='.', name='Cora')

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(hidden_channels, out_channels)
        )

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = self.mlp(x)
        return x

model = GCN(dataset.num_features, 16, dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

In [45]:
data = dataset[0]

In [46]:
train_mask = data.train_mask
test_mask = data.test_mask
val_mask = data.val_mask

In [47]:
verbose=1

In [48]:
for i in range(1, 201):
    model.train()
    optimizer.zero_grad()
    pred=model(data.x, data.edge_index)
    loss=criterion(pred[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()
    if i %10 ==0 and verbose > 0:
        print(f"Epoch: {i}; Training loss: {loss}")

    if i %10 ==0 and verbose > 0:
        model.eval()
        with torch.no_grad():
            pred=model(data.x, data.edge_index)
            loss=criterion(pred[val_mask], data.y[val_mask]) 
            print(f"Epoch: {i}; Val loss: {loss}")      

Epoch: 10; Training loss: 1.3323352336883545
Epoch: 10; Val loss: 1.486135482788086
Epoch: 20; Training loss: 0.28133058547973633
Epoch: 20; Val loss: 0.9605183601379395


Epoch: 30; Training loss: 0.01728327013552189
Epoch: 30; Val loss: 0.9047530889511108
Epoch: 40; Training loss: 0.0033787221182137728
Epoch: 40; Val loss: 0.9759785532951355
Epoch: 50; Training loss: 0.0027165785431861877
Epoch: 50; Val loss: 0.9365189075469971
Epoch: 60; Training loss: 0.0039945743046700954
Epoch: 60; Val loss: 0.8942247033119202
Epoch: 70; Training loss: 0.00594038050621748
Epoch: 70; Val loss: 0.8470942974090576
Epoch: 80; Training loss: 0.006958681624382734
Epoch: 80; Val loss: 0.8228334784507751
Epoch: 90; Training loss: 0.006824422162026167
Epoch: 90; Val loss: 0.8138474822044373
Epoch: 100; Training loss: 0.006348318420350552
Epoch: 100; Val loss: 0.8136910200119019
Epoch: 110; Training loss: 0.00593660119920969
Epoch: 110; Val loss: 0.8159820437431335
Epoch: 120; Training loss: 0.005590224638581276
Epoch: 120; Val loss: 0.817695140838623
Epoch: 130; Training loss: 0.005276016425341368
Epoch: 130; Val loss: 0.8190866708755493
Epoch: 140; Training loss: 0.0050402

In [49]:
model.eval()
with torch.no_grad():
    logits = model(data.x, data.edge_index)
    test_preds = logits[test_mask].argmax(dim=1)
    test_acc = (test_preds == data.y[test_mask]).float().mean().item()
print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 0.7800


In [50]:
# The performance is for with the mlp as without it 